In [1]:
"""
    ! this file is supposed to generate text data that contains these types of words:
        ? action: a verb that describes the action the robot is supposed to do (turn, move...)
        ? subject: how the robot is supposed to do the action (lift, square...)
        ? unit: if a measure of time or distance is needed 
        ? value: value of time of distance
"""

#! 
    #! The generated dataset will contain a lot of duplicates when the number of commands goes up, consider adding more more rules and using relatively small number
#!

'\n    ! this file is supposed to generate text data that contains these types of words:\n        ? action: a verb that describes the action the robot is supposed to do (turn, move...)\n        ? subject: how the robot is supposed to do the action (lift, square...)\n        ? unit: if a measure of time or distance is needed \n        ? value: value of time of distance\n'

In [ ]:

# from jetbot_mini.msg import Commands_msg
import tensorflow as tf
import numpy as np
import cv2
import mediapipe as mp

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm


In [ ]:
class generator:
    def __init__(self) -> None:
        
        self.actions = ['move', 'turn', 'advance', 'make a turn', 'make', 'keep moving', 'draw', 'write', 'move in the shape of', 'write the letter']

        self.subjects = ['forward', 'backward', 'left', 'right', 'circle', 'circle with radius','A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'right angle', 'left angle']

        self.units = ["meters", "seconds", 'degrees']

        self.values = np.arange(0, 100,0.25)

        self.action_classes = {"straight": ['move', 'advance', 'keep moving'],
                               "turns": ['turn', 'make a turn', 'make'],
                               "shapes": ['draw', 'write', 'move in the shape of', 'write the letter']}

        self.subject_classes = {"shapes": ['circle', 'circle with radius','A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'],
                                "directions":['forward', 'backward', 'left', 'right'],
                                "angles": ['right angle', 'left angle']}

        self.unit_classes = {"distance": ['m', 'meter', 'meters'],
                             "time": ['s', 'second', 'seconds', 'm', 'minute', 'minutes'],
                             "angle": ['d', 'deg', 'degree', 'degrees', 'r', 'rad', 'radian', 'radians']
                             }
        
        self.value_classes = {"value": self.values}


        self.rules = {"<START>,<A>straight": ['<S>directions'],
                      "<A>straight,<S>directions":['<V>value'],
                      "<S>directions,<V>Value": ['<U>time', '<U>distance'],
                      "<V>Value,<U>time":['<END>'],
                      "<V>Value,<U>distance":['<END>'],

                      "<START>,<A>turns": ["<S>directions(left|right)"],
                      "<A>turns,<S>directions(left|right)": ['<V>value'],
                      "<S>directions(left|right),<V>value":['<U>angle'],
                      "<V>value,<U>angle":['<END>'],

                      "<START>,<A>shapes": ['<S>shapes'],
                      "<A>shapes,<S>shapes": ["<END>"]}

        self.class_symbols = {"<A>": self.action_classes, "<S>": self.subject_classes, "<V>": self.value_classes, "<U>": self.unit_classes}

    def decode_token(self, token):
        if token == "<END>":
            return {"class": "deliminer", "value": 'end'}

        elif token == "<START>":
            return {"class": "deliminer", "value": 'start'}

        else:
            token_class_symbol = token[:3]
            token_class = self.class_symbols.get(token_class_symbol)
            value = token[3:]
            enumerations = []
            if '|' in token:
                    enumerations = token.split('(')[1].split(')')[0].split('|')
            return {"class": token_class, "value": value, "enumerations": enumerations}

    def get_value_from_token(self, token):
        if token != "<END>":
            token_class_symbol = token[:3]
            token_class = self.class_symbols.get(token_class_symbol)
            enumerations = []
            if '|' in token:
                enumerations = token.split('(')[1].split(')')[0].split('|')
            token = token[3:]
            token_values = token_class.get(token)
            random_value = str(np.random.choice(token_values)) if len(enumerations) == 0 else str(np.random.choice(enumerations))
            return random_value
        else:
            return ""
            # print(f"token: {token}")
            # print(f"token_class: {token_class}")
            # print(f"token values: {token_values}")
            # print(f"random values: {random_value}")

    def translate_descriptor(self, descriptor):
        text = []
        for token in descriptor:
            if token != "<END>":
                token_class_symbol = token[:3]
                token_class = self.class_symbols.get(token_class_symbol)
                enumerations = []

                if '|' in token:
                    enumerations = token.split('(')[1].split(')')[0].split('|')
                token = token[3:]

                token_values = token_class.get(token)

                random_value = str(np.random.choice(token_values)) if len(enumerations) == 0 else str(np.random.choice(enumerations))
                text.append(random_value)
                # print(f"token: {token}")
                # print(f"token_class: {token_class}")
                # print(f"token values: {token_values}")
                # print(f"random values: {random_value}")

        return " ".join(text)
                
        

    def generate_text(self, count):
        texts_descriptors = []
        texts = []
        for _ in tqdm(range(count), ncols=100):
            init_rule = np.random.choice([rule for rule in self.rules if rule.split(',')[0] == "<START>"])
            prev_token, current_token = init_rule.split(",")[0], init_rule.split(",")[1]
            
            

            next_token = ""
        
            text_descriptor = [current_token]

            add = True
            while next_token != "<END>":
                new_rule = ",".join([prev_token, current_token])
                matching_rules = self.rules.get(new_rule)
                if not matching_rules:
                    add = False
                    break
                
                next_token = np.random.choice(matching_rules)


                # text += self.get_value_from_token(next_token)

                #!###################################
                text_descriptor.append(next_token)
                #!###################################

                prev_token = current_token
                current_token = next_token
        

            if add:
                # self.translate_descriptor(text_describer)
                texts_descriptors.append(text_descriptor)
                texts.append(self.translate_descriptor(text_descriptor))
        return (texts_descriptors, texts)

In [ ]:
gen = generator()

generated = gen.generate_text(5000000)
descriptors, text = generated[0], generated[1]

In [ ]:
df = pd.DataFrame(columns=["descriptors", "text"])

df["descriptors"] = descriptors
df["text"] = text

In [ ]:
df.to_csv("./commands.csv")

In [ ]:
cmds = pd.read_csv("./commands.csv")

In [ ]:
cmds['text'].drop_duplicates(keep='first').shape

In [ ]:
with open("./unique.txt", "w") as unique_file:
    cmd = ""
    for text in cmds["text"].unique():
        cmd += text + ","

    unique_file.write(cmd)